In [8]:
from utils.preprocess import process_dataset
from utils.env_setup import get_stock_env
from environments.MutliAgentStockTradingEnv import MultiAgentStockTradingEnv
train, valid = process_dataset()

Successfully added technical indicators
[*********************100%%**********************]  1 of 1 completed
Shape of DataFrame:  (3310, 8)
Successfully added vix
Successfully added turbulence index


/home/abyaadrafid/anaconda3/envs/RLAC/lib/python3.10/site-packages/finrl/meta/preprocessor/preprocessors.py:106: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill").fillna(method="bfill")
/home/abyaadrafid/Desktop/TUM/thesis/utils/preprocess.py:36: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  processed_full = processed_full.fillna(method="bfill")


In [61]:
from typing import Dict, Optional

import gym
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
from ray.rllib.utils import override
import random

class MultiAgentStockTradingEnv(MultiAgentEnv, gym.Env):

    NAME = "MultiAgentStockTradingEnv"
    NUM_ACTIONS = None
    NUM_AGENTS = 2

    def __init__(self, env:DummyVecEnv):
        self.base_env = env
    
    @override(gym.Env)
    def step(self, actions:Dict, agent_id:list):
        state, rewards, dones = self.execute_actions(actions, agent_id)
        return state, rewards, dones, {}
    
    def execute_actions(self, actions:Dict, agent_id:list):
        random.shuffle(agent_id)
        
        dones = []
        rewards = []
        for agent in agent_id:
            state, reward, done, _ = self.base_env.step(actions[agent])
            rewards.append(reward)
            dones.append(done)
        return state, rewards, dones

In [62]:
env = get_stock_env(train)
marlenv = MultiAgentStockTradingEnv(env)

In [63]:
from stable_baselines3 import A2C
agent1 = A2C("MlpPolicy", env, verbose=1)
agent2 = A2C("MlpPolicy", env, verbose=1)

Using cpu device
Using cpu device


In [64]:
actions1, states = agent1.predict(env.reset())
actions2, states = agent2.predict(env.reset())

In [75]:
import numpy as np
actions=np.stack(([actions1,actions2]),axis=0)

In [77]:
actions[0].shape

(1, 29)

In [80]:
agent_id = [0,1]
states, rewards, dones, _ = marlenv.step(actions, agent_id)

In [34]:
random.shuffle(agent_id)

In [35]:
agent_id

[1, 0]

In [81]:
rewards

[array([0.01182447], dtype=float32), array([-0.32305568], dtype=float32)]